# Machine Learning
      ---Titanic/Regres_Logís
 - Prof. Neylson
 - Izabela Hendrix
 - Warley Gomes

### Classificação - Regressão Logística
   Vamos estimar modelos de regressão logística para enteder as chances de sobrevivência no Titanic
  a partir de atribuitos disponíveis

In [0]:
# Importar os módulos necessários
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from statsmodels.discrete.discrete_model import Logit
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score


In [3]:
bd = pd.read_csv("https://raw.githubusercontent.com/rebeccabilbro/titanic/master/data/train.csv")
bd.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


**Explicar sobrevivência pela idade, sexo e pela classe do passageiro:**





In [4]:
bd.dtypes # tipo das variáveis

PassengerId      int64
Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked        object
dtype: object

In [6]:
# Modificando a variável Pclass para categoria
bd["Pclass"] = bd["Pclass"].astype("category")
bd.dtypes

PassengerId       int64
Survived          int64
Pclass         category
Name             object
Sex              object
Age             float64
SibSp             int64
Parch             int64
Ticket           object
Fare            float64
Cabin            object
Embarked         object
dtype: object

In [7]:
# Montando o modelo
# Y = Survived
# X = Age, Sex, Pclass
# Preprocessing
bd1 = pd.get_dummies(bd, drop_first=True)
bd1.columns

Index(['PassengerId', 'Survived', 'Age', 'SibSp', 'Parch', 'Fare', 'Pclass_2',
       'Pclass_3', 'Name_Abbott, Mr. Rossmore Edward',
       'Name_Abbott, Mrs. Stanton (Rosa Hunt)',
       ...
       'Cabin_F G63', 'Cabin_F G73', 'Cabin_F2', 'Cabin_F33', 'Cabin_F38',
       'Cabin_F4', 'Cabin_G6', 'Cabin_T', 'Embarked_Q', 'Embarked_S'],
      dtype='object', length=1727)

In [21]:
# Criando as matrizes para a estimação
# Separando as variáveis de interesse
bd1 = bd1[["Age", "Sex_male", "Pclass_2","Pclass_3", "Survived"]]
# Remover os NAs
bd1 = bd1.dropna()
bd1.shape


(714, 5)

In [27]:
# Dividimos nossos dados 
y = bd1.Survived
X = bd1[["Age", "Sex_male", "Pclass_2","Pclass_3"]]
X.shape
y.shape


(714,)

In [29]:
X["intercept"]= 1
X.shape

(714, 5)

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3, random_state=1)

In [32]:
# Estimar o modelo e visualizar os resultados
model1 = Logit(y_train, X_train).fit()
print(model1.summary2())

Optimization terminated successfully.
         Current function value: 0.462052
         Iterations 6
                        Results: Logit
Model:              Logit            No. Iterations:   6.0000  
Dependent Variable: Survived         Pseudo R-squared: 0.320   
Date:               2019-03-25 23:22 AIC:              471.1283
No. Observations:   499              BIC:              492.1913
Df Model:           4                Log-Likelihood:   -230.56 
Df Residuals:       494              LL-Null:          -339.28 
Converged:          1.0000           Scale:            1.0000  
----------------------------------------------------------------
            Coef.   Std.Err.     z      P>|z|    [0.025   0.975]
----------------------------------------------------------------
Age        -0.0316    0.0090   -3.5110  0.0004  -0.0492  -0.0139
Sex_male   -2.5629    0.2455  -10.4413  0.0000  -3.0440  -2.0818
Pclass_2   -1.2833    0.3271   -3.9229  0.0001  -1.9245  -0.6422
Pclass_3   -2.3819   

In [37]:
# Calcular as chances relativas
np.exp(model1.params)


Age           0.968918
Sex_male      0.077081
Pclass_2      0.277114
Pclass_3      0.092376
intercept    36.844502
dtype: float64

- Os homens tem 07 porcentos da chance de sobrevivência das mulheres
- Para cada ano há mais Pclass2 vai ter 27 porcentoS das chances relativas da Pclass3
- Para cada ano há mais Pclass3 vai ter 9 porcento das chances relativas da Pclass3

In [0]:
def get_oddsperc(x):
    res = (np.exp(x) - 1) * 100
    return(res)

In [39]:
get_oddsperc(model1.params)

Age            -3.108190
Sex_male      -92.291929
Pclass_2      -72.288630
Pclass_3      -90.762383
intercept    3584.450154
dtype: float64

- Para cada um ano há mais na idade espera uma diminuição de 3 porcentos na chances de sobrevivencia.

- Homens têm menos 92 porcentos de chance de sobreviver do que as mulheres


In [43]:
# Predizendo a minha chance de sobrevivência
Warley = pd.DataFrame({"Age": 23, "Sex_male":1,
                       "Pclass_2":0, "Pclass_3":1, "intercept":1},
                      index=[0])
print(model1.predict(Warley))

0    0.3133
dtype: float64
